In [36]:
import numpy as np
import pandas as pd
import random
import itertools
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sklearn
from collections import defaultdict, Counter
from bisect import bisect_left
import matplotlib.pyplot as plt
import matplotlib
import io
import base64
import pymp
from multiprocessing import Pool, Manager, cpu_count
from IPython.display import display_html
from itertools import chain, cycle
import lightgbm as lgb
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import *
from src.util import *
from src import config
from src.visualize import *
from src.rewards import *
pd.set_option('display.float_format', lambda x: '%.02f' % x)

In [37]:
# https://www.kaggle.com/chasembowers/sequence-postprocessing-v2-67-lb/notebook#Sequence-Datasets

In [38]:
root = '../input/feedback-prize-2021'

In [39]:
df = pd.read_feather(f'{root}/train_en.fea')

In [40]:
MAX_SEQ_LEN = {}
df['len'] = df.end - df.start
max_lens = df.groupby('para_type')['len'].quantile(.995)
for i in range(len(ALL_CLASSES)):
  MAX_SEQ_LEN[id2dis[i]] = int(max_lens[i])
MAX_SEQ_LEN

{'Nothing': 132,
 'Claim': 55,
 'Evidence': 299,
 'Position': 59,
 'Concluding Statement': 191,
 'Lead': 197,
 'Counterclaim': 95,
 'Rebuttal': 116}

In [41]:
proba_thresh = {
    "Lead": 0.7,
    "Position": 0.55,
    "Evidence": 0.65,
    "Claim": 0.55,
    "Concluding Statement": 0.7,
    "Counterclaim": 0.5,
    "Rebuttal": 0.55,
    "Nothing": 0.5,
}
# TODO 自动学习这个阈值

In [42]:
d = pd.read_feather(f'{root}/para_label.fea')

In [43]:
gts = {}
for row in tqdm(d.itertuples(), total=len(d)):
  gts[row.id] = decode_label_all(row.start, row.para_types)

  0%|          | 0/15594 [00:00<?, ?it/s]

In [44]:
folds = {}
for row in tqdm(df.itertuples(), total=len(df)):
  folds[row.id] = row.kfold

  0%|          | 0/175160 [00:00<?, ?it/s]

In [45]:
np.random.seed(12345)

In [46]:
subfolds = {}
for row in tqdm(df.itertuples(), total=len(df)):
  subfolds[row.id] = np.random.randint(5)

  0%|          | 0/175160 [00:00<?, ?it/s]

In [47]:
gts['A8445CABFECE']

{0: [[0, 3]],
 1: [[15, 36]],
 2: [[36, 177]],
 3: [[3, 15]],
 4: [[177, 212]],
 5: [],
 6: [],
 7: []}

In [48]:
def para_dataset(info, disc_type=None, start_thre=0.1, start_thre2=0.99, fold=None, subfold=None, infer=False):
  total = len(info['id'])
  fes = []
  scores = []
  matches = []
  for i in tqdm(range(total)):
    x = {}
    for key in info:
      x[key] = info[key][i]
    id = x['id']
    gt = gts[id]
    fe = {}
    fe['id'] = id
    fe['index'] = i
    fe['fold'] = folds[id]
    if fold is not None and fe['fold'] != fold:
      continue
    fe['subfold'] = subfolds[id]
    if subfold is not None and fe['subfold'] != subfold:
      continue
    fe['num_words'] = x['num_words']
    fe['seps'] = (x['start_probs'][:,1] > 0.5).sum()
    fe['sep_ratio'] = fe['seps'] / fe['num_words']
    num_words = x['num_words']
    for j in range(num_words):
      if j > 0 and x['start_probs'][j][1] < start_thre and x['preds'][j] == x['preds'][j - 1]:
        continue
      probs = np.zeros_like(x['probs'][j])
      probs += x['probs'][j]
      fe['start'] = j
      fe['start_ratio'] = (j + 1) / num_words
      fe['start_probs'] = x['probs'][j]
      fe['start_sep_prob'] = x['start_probs'][j][1] if j > 0 else 1.
      preds = {k: 0 for k in range(NUM_CLASSES)}
      fe['start_cls'] = x['preds'][j]
      fe['max_start_prob'] = probs.max()
      preds[x['preds'][j]] += 1
      fe['pre_cls'] = -1 if j == 0 else x['preds'][j - 1]
      fe['pre_max_prob'] = 1 if j == 0 else x['probs'][j - 1].max()
      sep_count = 0
      for k in range(j + 1, num_words):
        probs += x['probs'][k]
        fe['para_len'] = k + 1 - j
        fe['para_len_ratio'] = (k + 1 - j) / num_words
        preds[x['preds'][k]] += 1
        if k + 1 < num_words and x['start_probs'][k + 1][1] < start_thre and x['preds'][k] == x['preds'][k + 1]:
          continue
        if k + 1 < num_words and x['start_probs'][k + 1][1] > 0.5:
          sep_count += 1
        fe['end'] = k + 1
        fe['end_ratio'] = (k + 1) / num_words
        fe['end_probs'] = x['probs'][k]
        fe['max_end_prob'] = x['probs'][k + 1].max() if k + 1 < num_words else 1
        end_cls = np.argmax(x['probs'][k])
        fe['end_cls'] = end_cls
        fe['end_sep_prob'] = x['start_probs'][k + 1][1] if k + 1 < num_words else 1.
        fe['sep_add_prob'] = (fe['start_sep_prob'] + fe['end_sep_prob']) / 2.
        fe['sep_mul_prob'] = (fe['start_sep_prob'] * fe['end_sep_prob']) ** 0.5
        fe['num_classes'] = len([k for k in range(NUM_CLASSES) if preds[k] > 0])
        mean_probs = gezi.softmax(probs)
        fe['mean_probs'] = mean_probs
        fe['max_prob'] = mean_probs.max()
        top_classes = np.argsort(-mean_probs,axis=0)[:2]
        fe['top_class'] = top_classes[0]
        fe['top_class2'] = top_classes[1]
        fe['next_cls'] = -1 if k + 1  == num_words else x['preds'][k + 1]
        # top_classes = top_classes[:1]
        if fe['para_len'] > 1 and fe['num_classes'] < 4:
          for cls in top_classes:
            if disc_type != None and cls != disc_type:
              continue
            # if cls and fe['para_len'] >  MAX_SEQ_LEN[id2dis[cls]]:
            #   continue
            # if fe['mean_probs'][cls] < proba_thresh[id2dis[cls]]:
            #   continue
            fe['mean_prob'] = fe['mean_probs'][cls]
            fe['start_prob'] = x['probs'][j][cls]
            fe['end_prob'] = x['probs'][k][cls]
            fe['class_ratio'] = (x['preds'][j:k+1] == cls).sum() / fe['para_len']
            # fe['class_max_prob'] = x['probs'][j:k+1][cls].max()
            # fe['class_min_prob'] = x['probs'][j:k+1][cls].min()
            fe['cls'] = cls
            fe['is_top_class'] = int(cls == fe['top_class'])
            fe['is_top_class2'] = int(cls == fe['top_class2'])
            if not infer:
              fe['score'] = best_match(gt[cls], [j, k + 1])
              fe['match'] = calc_match(gt[cls], [[j, k + 1]])
              # if not fe['match'] and np.random.randint(3) != 0:
              #   continue
              scores.append(fe['score'])
              matches.append(fe['match'])
            fes.append(fe.copy())
        if k + 1 < num_words and x['start_probs'][k + 1][1] > start_thre2:
          break
        if sep_count > 2:  
          break
  if not infer:
    ic(np.mean(scores), np.mean(matches))
  d = pd.DataFrame(fes)
  return d

In [ ]:
info = gezi.load('../working/offline/44/valid.pkl')

In [15]:
nw = 5
ds = Manager().dict()
with pymp.Parallel(nw) as p:
  for fold in p.range(5):
    fold_ = fold
    ds[fold_] = para_dataset(info, fold=fold)

  0%|          | 0/15594 [00:00<?, ?it/s]

ic| <ipython-input-13-c602e6a91567>:98 in para_dataset()
    npmean.(scores): 
0.2602468290317452    np.mean(matches): 0.24468753785262276
ic| -<ipython-input13-c602e6a91567>:98 in para_dataset()np
    .mean(scores): 0.26072600215658354
    np.mean(matches): 0.24603241455691266
ic| <ipython-input-13-c602e6a91567>:98 in para_dataset()
    np.mean(scores): 0.2607095447385953
    np.mean(matches): 0.24669412915271316
ic| <ipython-input--13c602e6a91567>:98 in para_dataset()
    np.mean(scores): 0.24418819056263794
    np.mean(:matches) 0.21973929728672542
ic| <ipython-input-13-c602e6a91567>:98 in para_dataset()
    np.mean(scores): 0.24742760015593704
    np.mean(matches): 0.23014723759194


In [16]:
dtrain = pd.concat([ds[0], ds[2], ds[3], ds[4]])

In [17]:
dtrain['match'] = dtrain['match'].apply(lambda x: min(x, 1))

In [18]:
dvalid = ds[1]

In [19]:
dvalid['match'] = dvalid['match'].apply(lambda x: min(x, 1))

In [20]:
# dtrain = d[d.subfold != 0]
# # dtrain = d0[d0.subfold == 1]
# dvalid = d[d.subfold == 0]
reg_cols =  [
              'num_words', 'start', 'start_ratio', 'end', 'end_ratio',
              'start_sep_prob', 'end_sep_prob', 'para_len', 'para_len_ratio',
              'num_classes', 'mean_prob', 
              # 'start_prob', 'end_prob', 
              # 'seps', 'sep_ratio', 
              'class_ratio',
              # 'sep_add_prob', 'sep_mul_prob', 
              # 'is_top_class', 'is_top_class2',
            #   'class_max_prob', 'class_min_prob'
        ]
cat_cols = [
            'cls', 
            # 'start_cls', 'end_cls',
            'pre_cls', 
            'next_cls'
            ]
label_col = 'match'
label_col = 'score'
cols = reg_cols + cat_cols
X_train = dtrain[cols]
y_train = dtrain[[label_col]]

In [21]:
X_valid = dvalid[cols]
# y_valid = dvalid[[label_col]]
y_valid = dvalid[['match']]

In [22]:
learning_rate = 0.1
num_boost_round = 1000
params = {
          # "objective": "binary",
          # "objective": "regression" if label_col is 'score' else 'binary',
          "objective": "cross_entropy",
          "metric": "auc",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": 12,
          "max_bin": 256,
          "feature_fraction": 0.75,
          "verbosity": 0,
          # "drop_rate": 0.1,
          "is_unbalance": True,
          # "max_drop": 50,
          "min_child_samples": 20,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "bagging_freq": 5,
          "bagging_fraction": 0.9,
          # "num_trees": 200,
          "subsample": 0.9
          }

In [23]:
d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid, y_valid, reference=d_train)
bst = lgb.train(params, d_train, num_boost_round, valid_sets=d_valid, 
                categorical_feature=cat_cols,
                verbose_eval=1,
                early_stopping_rounds=10)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.995625 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=0.9 will be ignored. Current value: bagging_fraction=0.9
[1]	valid_0's auc: 0.783747
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.799093
[3]	valid_0's auc: 0.796232
[4]	valid_0's auc: 0.795439
[5]	valid_0's auc: 0.79427
[6]	valid_0's auc: 0.79684
[7]	valid_0's auc: 0.806122
[8]	valid_0's auc: 0.807877
[9]	valid_0's auc: 0.808349
[10]	valid_0's auc: 0.80997
[11]	valid_0's auc: 0.809344
[12]	valid_0's auc: 0.811225
[13]	valid_0's auc: 0.811155
[14]	valid_0's auc: 0.812731
[15]	valid_0's auc: 0.813981


In [24]:
l = list(zip(bst.feature_name(), bst.feature_importance()))
l.sort(key=lambda x: -x[1])
l

[('cls', 426),
 ('num_words', 376),
 ('mean_prob', 334),
 ('next_cls', 220),
 ('end_sep_prob', 218),
 ('class_ratio', 218),
 ('para_len_ratio', 207),
 ('para_len', 164),
 ('pre_cls', 149),
 ('start_sep_prob', 131),
 ('start', 121),
 ('end_ratio', 120),
 ('start_ratio', 111),
 ('end', 77),
 ('num_classes', 76)]

In [25]:
def greedy_decodes(df, cols, para_classifier):
  proba_thresh = {
    "Lead": 0.99,
    "Position": 0.55,
    "Evidence": 0.7,
    "Claim": 0.55,
    "Concluding Statement": 0.99,
    "Counterclaim": 0.5,
    "Rebuttal": 0.5,
    "Nothing": 0.1,
  }

  df['pred'] = para_classifier.predict(df[cols])
  ids = set(df.id)
  ids_list, types_list, preds_list = [], [], []
  for id in tqdm(ids):
    d = df[df.id == id]
    d = d.sort_values(['pred'], ascending=[False])
    num_words = d.num_words.values[0]
    used = np.zeros(num_words)
    for row in d.itertuples():
      start, end = row.start, row.end
      cls = row.cls
      if used[start: end].sum() == 0:
        # used[start: end] = 1
        # if cls > 0:
        if row.mean_prob > proba_thresh[id2dis[cls]]:
          used[start: end] = 1
          if cls > 0:
            ids_list.append(id)
            types_list.append(id2dis[cls])
            preds_list.append(' '.join([str(x) for x in range(start, end)]))
      if used.sum() == num_words:
        break
          
  df = pd.DataFrame({
    'id': ids_list,
    'class': types_list,
    'predictionstring': preds_list,
  })
  return df

In [26]:
df_pred = greedy_decodes(dvalid, cols, bst)

  0%|          | 0/3050 [00:00<?, ?it/s]

In [27]:
df_gt = pd.read_csv('../working/offline/44/valid_gt.csv')

In [28]:
res = calc_metrics(df_gt[df_gt.id.isin(set(df_pred.id))], df_pred)
ic(res['f1/Overall'])
ic(res)

calc_f1:   0%|          | 0/7 [00:00<?, ?it/s]

ic| <ipython-input-28-cb78f7a414c1>:2 in <module>
    res['f1/Overall']: 0.7050391018600468
ic| <ipython-input-28-cb78f7a414c1>:3 in <module>
    res: {'acc/Claim': 0.7018239492466296,
          'acc/Concluding Statement': 0.866044776119403,
          'acc/Counterclaim': 0.6058879392212726,
          'acc/Evidence': 0.7858490566037736,
          'acc/Lead': 0.8424543946932007,
          'acc/Position': 0.7454798331015299,
          'acc/Rebuttal': 0.50920245398773,
          'cm/Claim': array([[6195, 2632],
                [3619, 5208]]),
          'cm/Concluding Statement': array([[2321,  359],
                [ 312, 2368]]),
          'cm/Counterclaim': array([[638, 415],
                [523, 530]]),
          'cm/Evidence': array([[6664, 1816],
                [2317, 6163]]),
          'cm/Lead': array([[1524,  285],
                [ 299, 1510]]),
          'cm/Position': array([[2144,  732],
                [ 874, 2002]]),
          'cm/Rebuttal': array([[415, 400],
             

{'f1/Overall': 0.7050391018600468,
 'f1/Claim': 0.6646639128802103,
 'acc/Claim': 0.7018239492466296,
 'recall/Claim': 0.6312410841654779,
 'cm/Claim': array([[6195, 2632],
        [3619, 5208]]),
 'f1/Evidence': 0.7633010709581353,
 'acc/Evidence': 0.7858490566037736,
 'recall/Evidence': 0.7420109119251753,
 'cm/Evidence': array([[6664, 1816],
        [2317, 6163]]),
 'f1/Position': 0.7275195113674924,
 'acc/Position': 0.7454798331015299,
 'recall/Position': 0.7104042412193505,
 'cm/Position': array([[2144,  732],
        [ 874, 2002]]),
 'f1/Concluding Statement': 0.8737060041407867,
 'acc/Concluding Statement': 0.866044776119403,
 'recall/Concluding Statement': 0.8815039878465628,
 'cm/Concluding Statement': array([[2321,  359],
        [ 312, 2368]]),
 'f1/Lead': 0.8392070484581498,
 'acc/Lead': 0.8424543946932007,
 'recall/Lead': 0.8359846407021393,
 'cm/Lead': array([[1524,  285],
        [ 299, 1510]]),
 'f1/Counterclaim': 0.5763324299909666,
 'acc/Counterclaim': 0.6058879392212

In [29]:
# df_pred2[df_pred2.id=='4125C404E34B']

In [30]:
# dvalid[dvalid.id=='4125C404E34B'][['id', 'pred', 'match', 'score', 'start', 'end', 'cls', 'mean_prob', 'start_sep_prob', 'end_sep_prob']]

In [31]:
# gts['4125C404E34B']

In [32]:
# ALL_CLASSES

In [33]:
# df_pred2 = df_pred2[df_pred2.id.isin(set(df_pred.id))]

In [34]:
# df_pred_fold1 = greedy_decodes(ds[1], cols, bst)

In [35]:
# res = calc_metrics(df_gt[df_gt.id.isin(set(df_pred_fold1.id))], df_pred_fold1)
# ic(res)